# Import the needed libraries

In [41]:
import numpy as np
import pandas as pd

# Entropy Implementation

In [42]:
def entropy(class_y):
    """ 
    Input: 
        - class_y: list of class labels (0's and 1's)
    
    Output: 
        - The entropy
        
    Compute the entropy for a list of classes
    """
    
    # Handling if there is only 1 or 0 labels
    if len(class_y) <= 1: 
        return 0
    
    # count
    total_count = np.bincount(class_y)
    # Find the probabilities
    probabilities = total_count[np.nonzero(total_count)] / len(class_y) 

    # Handling if the length of the probabilities is less than or equal to 1
    if len(probabilities) <= 1 : 
        return 0
    
    # Entropy equation
    return - np.sum(probabilities * np.log(probabilities)) / np.log(len(probabilities)) 

# Information Gain

In [43]:
def information_gain(previous_y, current_y):
    """
    Inputs:
        - previous_y : the distribution of original labels (0's and 1's)
        - current_y  : the distribution of labels after splitting based on a particular
                     split attribute and split value
    
    Outputs:
        - info_gain : The information gain after partitioning
        
    Compute and return the information gain from partitioning the previous_y labels into the current_y labels.
    
    """ 
    
    # Y = previous_y
    # X = current_y
    # IG = H(Y) - CE(Y|X)
    conditional_entropy = 0 
    for y in current_y:
        conditional_entropy += (entropy(y)*len(y)/len(previous_y))

    info_gain = entropy(previous_y) - conditional_entropy
    return info_gain

# The Decision Tree ID3 algorithm
1. Find best feature
2. Find best split for the best feature
3. Parition classes based on steps 1 and 2

# Step 3 : Partition Classes

In [44]:
def partition_classes(X, y, split_attribute, split_val):
    """
    Inputs:
    - X               : (N,D) list containing all data attributes
    - y               : a list of labels
    - split_attribute : column index of the attribute to split on
    - split_val       : either a numerical or categorical value to divide the split_attribute
    
    Outputs:
    - X_left          : X left after partitioning
    - X_right         : X right after partitioning
    - y_left          : y left after partitioning
    - y_right         : y right after partitioning
    
    Partition the data(X) and labels(y) based on the split value    
    """
    """
    Example:
    
    X = [[3, 'aa', 10],                 y = [1,
         [1, 'bb', 22],                      1,
         [2, 'cc', 28],                      0,
         [5, 'bb', 32],                      0,
         [4, 'cc', 32]]                      1]
    
    Here, columns 0 and 2 represent numeric attributes, while column 1 is a categorical attribute.
    
    Consider the case where we call the function with split_attribute = 0 (the index of attribute) and split_val = 3 (the value of attribute).
    Then we divide X into two lists - X_left, where column 0 is <= 3 and X_right, where column 0 is > 3.
    
    X_left = [[3, 'aa', 10],                 y_left = [1,
              [1, 'bb', 22],                           1,
              [2, 'cc', 28]]                           0]
              
    X_right = [[5, 'bb', 32],                y_right = [0,
               [4, 'cc', 32]]                           1]

    Consider another case where we call the function with split_attribute = 1 and split_val = 'bb'
    Then we divide X into two lists, one where column 1 is 'bb', and the other where it is not 'bb'.
        
    X_left = [[1, 'bb', 22],                 y_left = [1,
              [5, 'bb', 32]]                           0]
              
    X_right = [[3, 'aa', 10],                y_right = [1,
               [2, 'cc', 28],                           0,
               [4, 'cc', 32]]                           1]
               
    """
    
    X = np.array(X)
    column_split = X[:, split_attribute]
    X_left = []
    y_right = []
    X_right = []
    y_left = []
    
    # Counter for appending
    counter = 0

    # Numerical attribute
    if isinstance(split_val,str) == False: 
        for i in column_split:
            if i <= split_val:
                X_left.append(X[counter])
                y_left.append(y[counter])
            else:
                X_right.append(X[counter])
                y_right.append(y[counter])
            counter += 1

    # Categorical attribute    
    else: 
        for i in column_split:
            # == and NOT <=
            if i == split_val:
                X_left.append(X[counter])
                y_left.append(y[counter])
            else:
                X_right.append(X[counter])
                y_right.append(y[counter])
            counter += 1

    return X_left, X_right, y_left, y_right 

# Step 2 : Find Best Split

In [45]:
def find_best_split(X, y, split_attribute):
    
    """
    Inputs:
        - X               : (N,D) list containing all data attributes
        - y               : a list array of labels
        - split_attribute : Column of X on which to split
    
    Outputs:
        - best_split_val  : optimal split value for a given attribute
        - best_info_gain  : the corresponding information gain
    
    Compute and return the optimal split value for a given attribute, along with the corresponding information gain

    """
    
    """
    
      Example:
    
        X = [[3, 'aa', 10],                 y = [1,
             [1, 'bb', 22],                      1,
             [2, 'cc', 28],                      0,
             [5, 'bb', 32],                      0,
             [4, 'cc', 32]]                      1]
    
        split_attribute = 0
        
        Starting entropy: 0.971
        
        Calculate information gain at splits:
           split_val = 1  -->  info_gain = 0.17
           split_val = 2  -->  info_gain = 0.01997
           split_val = 3  -->  info_gain = 0.01997
           split_val = 4  -->  info_gain = 0.32
           split_val = 5  -->  info_gain = 0
        
       best_split_val = 4; info_gain = .32; 
    """
    
    # Initialize the best information gain
    best_info_gain = 0 
    X = np.array(X)
    column_split = X[:,split_attribute]

    # Get the unique values only !
    column_split = np.unique(column_split)
    # Set the best split as the initial one (in case it is the only one)
    best_split_val = column_split[0] 

    for split_val in column_split:
        # Call the partition classes function
        current_X_left, current_X_right, current_y_left, current_y_right = partition_classes(X, y, split_attribute, split_val)
        current_y = []
        current_y.append(current_y_left)
        current_y.append(current_y_right)

        # Calculate the information gain
        current_info_gain = information_gain(y,current_y)
        # check if it is better than before
        if current_info_gain > best_info_gain:
            best_info_gain = current_info_gain
            best_split_val = split_val

    return best_split_val, best_info_gain

# Step 1 : Find Best Feature

In [46]:
def find_best_feature(X, y):
    """
    Inputs:
        - X              : (N,D) list containing all data attributes
        - y              : a list of labels
    
    Outputs:
        - best_feature   : Best Feature to split on
        - best_split_val : Best Split Value for this feature
        
    Compute and return the optimal attribute to split on and optimal splitting value
    """
    
    """
        
    Example:
    
        X = [[3, 'aa', 10],                 y = [1,
             [1, 'bb', 22],                      1,
             [2, 'cc', 28],                      0,
             [5, 'bb', 32],                      0,
             [4, 'cc', 32]]                      1]
    
        split_attribute = 0
        
        Starting entropy: 0.971
        
        Calculate information gain at splits:
           feature 0:  -->  info_gain = 0.32
           feature 1:  -->  info_gain = 0.17
           feature 2:  -->  info_gain = 0.4199
        
       best_split_feature: 2 best_split_val: 22
    """
    
    
    best_info_gain = 0
    best_feature = 0
    best_split_val = 0
    # Loop over the features , find the best split 
    for feature_index in range(len(X[0])):
        current_best_split_val, current_best_info_gain =  find_best_split(X, y,  feature_index)
        if current_best_info_gain > best_info_gain:
            best_info_gain = current_best_info_gain
            best_feature = feature_index
            best_split_val = current_best_split_val
    return best_feature, best_split_val

# Decision Tree ID3 Full Algorithm : Putting Everything Together

In [47]:
class MyDecisionTree(object):
    def __init__(self, max_depth=None):
        """        
        Args:
        
        max_depth: maximum depth of the tree including the root node.
        """
        self.tree = {}
        # For prediction
        self.residual_tree = {}
        self.max_depth = max_depth

        
    def fit(self, X, y, depth):
        """
        Args:
        X     : N*D matrix corresponding to the data points
        Y     : N*1 array corresponding to the labels of the data points
        depth : depth of node of the tree
        
        Output:
        node : A dictionary that contains this node information
        """

        # Base cases
        unique_labels = np.unique(y) 
        # If we reach the maximum depth or the unique labels = 1
        if (len(unique_labels) == 1) or (depth == self.max_depth):
            unique_labels, counts_unique_labels = np.unique(y, return_counts=True)
            index = counts_unique_labels.argmax()
            classification = unique_labels[index]    
            return classification
    
        # Find best feature (Step 1)
        best_feat, best_split  = find_best_feature(X, y)
        # Split on this feature (Step 2) - (Redundant)
        best_split, information_gain = find_best_split(X, y, best_feat)
        # Partition on the best feature and split (Step 3)
        X_left, X_right, y_left, y_right = partition_classes(X, y, best_feat, best_split)
        
        if isinstance(best_split,str):
            # Represnt the sub-tree as a question and an answer
            question = "{} == {}".format(best_feat, best_split)
        else:
            # Represnt the sub-tree as a question and an answer
            question = "{} <= {}".format(best_feat, best_split)
        node = {question: []}
        
        # Find answers (recursion)
        depth+=1 
        # RECURSION on the left sub tree
        yes_answer = self.fit(X_left,y_left, depth)
        # RECURSION on the right sub tree
        no_answer = self.fit(X_right, y_right, depth)
        
        # Both trees are the same
        if yes_answer == no_answer:
            node = yes_answer
        else:
            # Append the questions to the answers
            node[question].append(yes_answer)
            node[question].append(no_answer)
        # The tree is equal to the node
        self.tree = node
        return node
        
    def predict(self, record,flag=1):
        """
        Args:
        
        record: D*1, a single data point that should be classified
        
        Output:
        prediction: True if the predicted class label is 1, False otherwise      
        
        classify a sample in test data set using self.tree and return the predicted label
        """
        # First time
        if flag == 1:
            self.residual_tree = self.tree
        question = list(self.residual_tree.keys())[0]
        # Split the question to get the feature and its value
        feature, comparison, value = question.split()

        
        # Check if str or int
        # String
        if comparison == "==": 
            if record[int(feature)] == value:
                # Left sub tree
                answer = self.residual_tree[question][0]
            else:
                # Right sub-tree
                answer = self.residual_tree[question][1]
        # Integer
        elif comparison == "<=": 
            if record[int(feature)] <= float(value):
                # Right Sub-tree
                answer = self.residual_tree[question][0]
            else:
                # Left sub-tree
                answer = self.residual_tree[question][1]

        # Base case
        # If we have the answer
        if not isinstance(answer, dict):
            return answer
    
        # Recursion
        else:
            # The residual tree is the answer!
            self.residual_tree = answer
            # have flag = 0 so the residual tree is our sub-problem
            return self.predict(record,0)